In [0]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
import re
import string
import nltk
import warnings
from nltk.corpus import stopwords
warnings.filterwarnings("ignore", category=DeprecationWarning)
from nltk.tokenize import *
nltk.download("popular")
from nltk.stem import WordNetLemmatizer

import textblob
from textblob import TextBlob

import nltk
nltk.download('punkt')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
pip install --upgrade google-cloud-bigquery

Requirement already up-to-date: google-cloud-bigquery in /usr/local/lib/python3.6/dist-packages (1.24.0)


In [0]:
credentials = service_account.Credentials.from_service_account_file(
    '/content/drive/My Drive/ML_works/Internet_sathi/IS_ML_PROBLEMS/infra-211714-fa79bf90e271.json')
project_id = 'infra-211714'
client = bigquery.Client(credentials= credentials,project=project_id)

In [0]:
original_data= client.query('''
SELECT * FROM `infra-211714.is_playstore.p_Reviews_IS` ''').result().to_dataframe()

In [0]:
pip install nltk

In [0]:
data=original_data[["Review_Text","Star_Rating"]].dropna()

In [0]:
data.isnull().sum()

Review_Text    0
Star_Rating    0
dtype: int64

In [0]:
data.Review_Text

1                internet saaythi
4                     Fully faltu
5                    kavita Dawar
9                  Internetsaathi
20                         गुड ऐप
                  ...            
889           best aap in program
891              lnterneretsaathi
892                          nice
896                      best aap
897    inter net Sathi option set
Name: Review_Text, Length: 250, dtype: object

In [0]:
#Removing punctuation

In [0]:
import string
def remove_punctuation(text):
  no_punct="".join([c for c in text if c not in string.punctuation])
  return no_punct

In [0]:
data["review_text"]=data.Review_Text.apply(lambda x:remove_punctuation(x))

In [0]:
data.head()

,Review_Text,Star_Rating,review_text
1,internet saaythi,4,internet saaythi
4,Fully faltu,1,Fully faltu
5,kavita Dawar,5,kavita Dawar
9,Internetsaathi,1,Internetsaathi
20,गुड ऐप,5,गुड ऐप


In [0]:
#tokenizing the words....
tokenizer=RegexpTokenizer(r'\w+')

In [0]:
data["review_text"]=data["review_text"].apply(lambda x:tokenizer.tokenize(x.lower()))

In [0]:
data.review_text.head()

1     [internet, saaythi]
4          [fully, faltu]
5         [kavita, dawar]
9        [internetsaathi]
20             [ग, ड, ऐप]
Name: review_text, dtype: object

In [0]:
sw=stopwords.words("english")

In [0]:
def remove_stopwords(text):
  a=[word for word in text if word not in sw]
  return a

In [0]:
data["review_text"]=data["review_text"].apply(lambda x:remove_stopwords(x))

In [0]:
data.review_text.head()

1     [internet, saaythi]
4          [fully, faltu]
5         [kavita, dawar]
9        [internetsaathi]
20             [ग, ड, ऐप]
Name: review_text, dtype: object

In [0]:
#Lematizer
lematizer=WordNetLemmatizer()

In [0]:
def lematizer_fun(text):
  lem=[lematizer.lemmatize(i) for i in text]
  return lem

In [0]:
data["review_text"]=data["review_text"].apply(lambda x:lematizer_fun(x))

In [0]:
#stemming

from nltk.stem.porter import *
stemmer = PorterStemmer()

In [0]:
#This function will do stemming of all the words present in the tweet.
def stem(x):
  el=[]
  for m in (x):
    el.append(stemmer.stem(m))
  return " ".join(el)

In [0]:
data["review_text"]=data["review_text"].apply(lambda x:stem(x))

In [0]:
data["sentiment"]=data.Review_Text.apply(lambda x:TextBlob(x).sentiment[0])
data["polarity"]=data.Review_Text.apply(lambda x:TextBlob(x).sentiment[1])

In [0]:
data.sentiment.mean()

0.33228523809523847

In [0]:
len(data)

250